In [9]:
import pandas as pd
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD
from keras.layers import BatchNormalization
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator

#####
# Ayden Shankman
# ECE-472
# Assigment 4
#####
# Got help from https://www.tensorflow.org/tutorials/images/cnn\


# Get top-1 and top-k accuracy
def top_k_accuracy(k, x_test, y_test, model):
    top_1_correct = 0
    top_k_correct = 0
    p = model.predict(x_test)
    for i, prob in enumerate(p):
        if(y_test[i][0] in np.argsort(prob)[-1:]):
            top_1_correct += 1
        if(y_test[i][0] in np.argsort(prob)[-k:]):
            top_k_correct += 1
    print("---------------------------------------------------")
    print("Top-1 Accuracy: " +  str(top_1_correct/len(x_test)))
    print("Top-" + str(k) + " Accuracy: " +  str(top_k_correct/len(x_test)))

# Formatting and Normalizing Data
def format_data(dataset):
    train_img, Y_train, test_img, y_test = [[0],[0],[0],[0]]
    if(dataset == 'cifar10'):
        (train_img, Y_train), (test_img, y_test) = datasets.cifar10.load_data()
    if(dataset == 'cifar100'):
        (train_img, Y_train), (test_img, y_test) = datasets.cifar100.load_data()

    # Normalize pixel values between 0-1 instead of 0-255
    X_train, x_test = train_img/255.0, test_img/255.0

    # Split training into training and validation
    x_train = X_train[0:int(len(X_train)-len(X_train)/5)]
    x_val = X_train[int(len(X_train)-len(X_train)/5):len(X_train)]
    y_train = Y_train[0:int(len(Y_train)-len(Y_train)/5)]
    y_val = Y_train[int(len(Y_train)-len(Y_train)/5):len(Y_train)]

    return x_train, y_train, x_val, y_val, x_test, y_test



# Building and Training Model
def train_model(x_train, y_train, x_val, y_val, type, name, epochs):
    print(y_train)
    # Data augmentation. Help from https://stepup.ai/train_data_augmentation_keras/
    datagen = ImageDataGenerator(
            featurewise_center=False, 
            samplewise_center=False,
            featurewise_std_normalization=False,
            samplewise_std_normalization=False,
            zca_whitening=False,
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images horizontally
            vertical_flip=False)

    datagen.fit(x_train)

    model = Sequential()
    model.add(layers.Conv2D(64, (3,3), padding = "same", kernel_initializer='he_uniform', activation = 'relu', input_shape = (32,32,3)))
    model.add(BatchNormalization())
    model.add(layers.Conv2D(64, (3,3), padding = "same", kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(layers.MaxPool2D(2,2))
    model.add(layers.Dropout(0.05))

    model.add(layers.Conv2D(128, (3,3), padding = "same", kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding = "same", kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(layers.MaxPool2D(2,2))
    model.add(layers.Dropout(0.1))

    model.add(layers.Conv2D(128, (3,3), padding = "same", kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(layers.Conv2D(128, (3,3), padding = "same", kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(layers.MaxPool2D(2,2))
    model.add(layers.Dropout(0.2))

    model.add(layers.Flatten())
    model.add(layers.Dense(128, kernel_initializer='he_uniform', activation = 'relu'))
    model.add(BatchNormalization())
    model.add(layers.Dropout(0.2))

    if(type == 'cifar10'):        
        model.add(layers.Dense(10, activation = 'softmax'))

    elif(type == 'cifar100'):
        model.add(layers.Dense(100, activation = 'softmax'))

    print(model.summary())

    adam = Adam(learning_rate = 0.001)

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer= adam,
                  metrics=['accuracy','sparse_top_k_categorical_accuracy'])

    history = model.fit(datagen.flow(x_train, y_train, batch_size=64), 
                        steps_per_epoch = x_train.shape[0] // 64, 
                        validation_data=(x_val, y_val),                
                        epochs=epochs, 
                        verbose = 1)

    model.save(name)
    return model, history



if __name__ == "__main__":

    x_train_cifar10, y_train_cifar10, x_val_cifar10, y_val_cifar10, x_test_cifar10, y_test_cifar10 = format_data('cifar10')
    x_train_cifar100, y_train_cifar100, x_val_cifar100, y_val_cifar100, x_test_cifar100, y_test_cifar100 = format_data('cifar100')

    train = False
    if(train):
        model,history = train_model(x_train_cifar100, y_train_cifar100, x_val_cifar100, y_val_cifar100, 'cifar100', name = 'model_cifar100_2.h5', epochs = 50)
        top_k_accuracy(5, x_test_cifar100, y_test_cifar100, model)
    else:
        # Cifar10 dataset
        print("\n\n\n----- MODEL FOR CIFAR10 -----")
        model_cifar10 = keras.models.load_model('model_cifar10.h5')
        print(model_cifar10.summary())
        top_k_accuracy(5, x_test_cifar10, y_test_cifar10, model_cifar10)
        
        # Cifar100 dataset
        print("\n\n\n----- MODEL FOR CIFAR100 -----")
        model_cifar100 = keras.models.load_model('model_cifar100.h5')
        print(model_cifar100.summary())
        top_k_accuracy(5, x_test_cifar100, y_test_cifar100, model_cifar100)





----- MODEL FOR CIFAR10 -----
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 64)        1792      
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        36928     
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 64)       256       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 16, 16, 64)       0         
 )                                                               
                       

First model I tried was first 2 convolutionary layers with filter size 32 then a maxpooling layer. I then repeated this again but with filter size 64 then with 128 followed by a dense layer of 128.

Next iteration I added Batch normalization after each double layer of convolution right before the max pooling layer and also after the 128 dense layer. This gave me higher accuracy.

Next iteration I changed the padding on the convolutions from 'valid' to 'same' and made the kernal initializer 'he_uniform'. This provided a slightly higher accuracy.

Next iteration I increased the dropout rate after each layer, to negate some of the over-fitting, and added an extra level of batch normalization in between each double convolution. Once again this raised the accuracy.

Finally, I changed the patter of convolution from 32, 64, 128 to 64, 64, 128. After training this model for 50 epochs with a learning rate of .001, I was able to achieve over 87% accuracy for the cifar10 dataset and a 88% top-5 accuracy for the cifar100 dataset. These were the highest accuracies I have achieved.